# Connecting to and starting TigerGraph service
Make sure to increase RAM available first

```ssh -p 14022 tigergraph@localhost```

Password: tigergraph

Once sshed into the container, run

```gadmin start all```

In [1]:
import pandas as pd
import pyTigerGraph as tg

cloud = True

In [2]:
if cloud:
    conn = tg.TigerGraphConnection(host="https://syntheapytg.i.tgcloud.io", useCert=True, gsqlVersion="3.0.5")
else:
    conn = tg.TigerGraphConnection(host="http://localhost")

In [3]:
print(conn.gsql("DROP ALL", options=[]))

Trying version: v3_0_5
Connecting to syntheapytg.i.tgcloud.io:14240
If there is any relative path, it is relative to <System.AppRoot>/dev/gdk/gsql
Dropping all, about 1 minute ...
Abort all active loading jobs
Resetting GPE...
Successfully reset GPE
Stopping GPE GSE
Successfully stopped GPE GSE in 0.041 seconds
Clearing graph store...
Successfully cleared graph store
Everything is dropped.



In [4]:
conditions = pd.read_csv("data/conditions.csv", names=["dateStart", "dateEnd", "patient", "condition", "code", "description"])
medications = pd.read_csv("data/medications.csv", names=["dateStart", "dateEnd", "patient", "medication", "code" , "description", "cost", "dispenses", "totalCost", "reasonCode", "reasonDesc"])
patients = pd.read_csv("data/patients.csv", names=["id","birthDate","deathDate","ssn","drivers","passport","prefix","first","last","suffix","maiden","marital","race","ethnicity","gender","birthPlace","address","city","state","zip"])
procedures = pd.read_csv("data/procedures.csv", names=["date", "patient", "encounter", "code", "description", "cost", "reasonCode", "reasonDescription"])
providers = pd.read_csv("data/providers.csv", names=["id","organization","name","gender","specialty","address","city","state","zip","utilization"])
encounters = pd.read_csv("data/encounters.csv", names=["id","dateStart","dateStop","patient","provider","encounterClass","code","description","cost","reasonCode","reasonDescription"])

In [5]:
patients.head()

,id,birthDate,deathDate,ssn,drivers,passport,prefix,first,last,suffix,maiden,marital,race,ethnicity,gender,birthPlace,address,city,state,zip
0,35e0c202-7d79-40cb-84f1-1b69e6f6be3c,1985-12-17,NaN,999-59-3229,S99985207,X13132187X,Mr.,Ignacio928,Candelaria844,NaN,NaN,M,black,dominican,M,Portsmouth Saint John Parish DM,940 Cummerata Pathway Apt 23,Worcester,Massachusetts,1545.0
1,ffd2b713-518b-4660-b112-e1ee298ac8d1,1990-10-20,NaN,999-77-6505,S99942717,X36715867X,Mr.,Robbie31,Parisian75,NaN,NaN,M,black,west_indian,M,Cheshire Massachusetts US,698 Cummerata Rapid,Boston,Massachusetts,2108.0
2,5311bb0b-a96c-4bab-8713-bc9405c420ee,1983-05-03,NaN,999-50-9809,S99995282,X64495409X,Mr.,Sergio619,Dicki44,NaN,NaN,M,white,english,M,Malden Massachusetts US,761 Lubowitz Terrace,Malden,Massachusetts,2148.0
3,81b0149c-1b72-4e63-9ee4-62e4856fdd5c,1953-07-16,NaN,999-12-3651,S99948962,X57816876X,Ms.,Cindie288,Hyatt152,NaN,NaN,S,white,french,F,Waltham Massachusetts US,1066 Weimann Extension,Brookline,Massachusetts,2215.0
4,d1f14152-8766-40cc-a53b-9e78aba41bc0,2002-08-02,NaN,999-45-4207,S99955456,NaN,NaN,Hsiu839,Rutherford999,NaN,NaN,NaN,white,french,F,Worcester Massachusetts US,622 Breitenberg Club Unit 79,Lowell,Massachusetts,1850.0


In [6]:
providers.head()

,id,organization,name,gender,specialty,address,city,state,zip,utilization
0,ef58ea08-d883-3957-8300-150554edc8fb,53a17430-bbb3-4c12-bf40-431a79de2764,Noe500 Dibbert990,M,GENERAL PRACTICE,60 HOSPITAL ROAD,LEOMINSTER,MA,1453,3585
1,69176529-fd1f-3b3f-abce-a0a3626769eb,b59e8b09-1421-43b6-badc-9bb317f744c1,Mariam937 Gleason633,F,GENERAL PRACTICE,330 MOUNT AUBURN STREET,CAMBRIDGE,MA,2138,4711
2,5e765f2b-e908-3888-9fc7-df2cb87beb58,0dda7391-f13e-4081-9177-14e2deaa0885,Dagny669 Schoen8,F,GENERAL PRACTICE,211 PARK STREET,ATTLEBORO,MA,2703,7141
3,f1fbcbfb-fcfa-3bd2-b7f4-df20f1b3c3a4,07ffa70e-326a-4f88-a6b0-4f3009e4171a,Tyron580 Torphy630,M,GENERAL PRACTICE,ONE GENERAL STREET,LAWRENCE,MA,1842,5455
4,e002090d-4e92-300e-b41e-7d1f21dee4c6,26c6423a-1f26-4aef-9d69-fa9aa0cf7737,Loren192 Fay398,M,GENERAL PRACTICE,1493 CAMBRIDGE STREET,CAMBRIDGE,MA,2138,4715


In [7]:
conditions.head()

,dateStart,dateEnd,patient,condition,code,description
0,1992-10-26,NaN,ffd2b713-518b-4660-b112-e1ee298ac8d1,848ce5a7-49d4-4d16-a07b-9c9efd3bae13,446096008,Perennial allergic rhinitis
1,2012-04-16,2012-06-22,ffd2b713-518b-4660-b112-e1ee298ac8d1,b08c099f-17d4-4789-adc3-63a4d3562f57,30832001,Rupture of patellar tendon
2,2012-08-26,2012-09-16,ffd2b713-518b-4660-b112-e1ee298ac8d1,b895cc31-dbcb-4ae9-926e-8103f61e605c,444814009,Viral sinusitis (disorder)
3,2015-01-14,2015-01-28,35e0c202-7d79-40cb-84f1-1b69e6f6be3c,367aa2ee-6416-46b3-b645-12f693ae57d1,283371005,Laceration of forearm
4,2013-11-21,2013-12-12,5311bb0b-a96c-4bab-8713-bc9405c420ee,522fbd8c-652b-4f19-b58a-3504f2bc7ecd,75498004,Acute bacterial sinusitis (disorder)


In [8]:
medications.head()

,dateStart,dateEnd,patient,medication,code,description,cost,dispenses,totalCost,reasonCode,reasonDesc
0,1992-02-27,NaN,ffd2b713-518b-4660-b112-e1ee298ac8d1,3348b59d-e0e4-4801-9e2a-ba3358869a05,665078,Loratadine 5 MG Chewable Tablet,11.58,331,3832.98,NaN,NaN
1,1992-02-27,NaN,ffd2b713-518b-4660-b112-e1ee298ac8d1,3348b59d-e0e4-4801-9e2a-ba3358869a05,1870230,NDA020800 0.3 ML Epinephrine 1 MG/ML Auto-Inje...,386.35,331,127881.85,NaN,NaN
2,1984-04-15,NaN,5311bb0b-a96c-4bab-8713-bc9405c420ee,d2a76b2a-e58d-4d0c-91e3-e7819ea6dcb1,477045,Chlorpheniramine Maleate 2 MG/ML Oral Solution,7.46,427,3185.42,NaN,NaN
3,1986-05-09,NaN,5311bb0b-a96c-4bab-8713-bc9405c420ee,1c9c93fd-74dd-4b6d-8501-885c32f508b0,895994,120 ACTUAT Fluticasone propionate 0.044 MG/ACT...,44.03,402,17700.06,233678006.0,Childhood asthma
4,1986-05-09,NaN,5311bb0b-a96c-4bab-8713-bc9405c420ee,1c9c93fd-74dd-4b6d-8501-885c32f508b0,745679,200 ACTUAT Albuterol 0.09 MG/ACTUAT Metered Do...,73.05,402,29366.10,233678006.0,Childhood asthma


In [9]:
procedures.head()

,date,patient,encounter,code,description,cost,reasonCode,reasonDescription
10,2009-08-20,81b0149c-1b72-4e63-9ee4-62e4856fdd5c,09a28878-7978-4989-ac3d-4928e442e089,428191000124101,Documentation of current medications,492.65,NaN,NaN
15,2011-11-12,d1f14152-8766-40cc-a53b-9e78aba41bc0,e6f00c20-12dc-4865-94a9-78a537f6ec0b,117015009,Throat culture (procedure),2282.58,43878008.0,Streptococcal sore throat (disorder)
47,2010-07-03,3508df6f-ed81-4d4a-8c4e-55494674bb49,50bb18ec-e46d-4cda-b748-c224a27e517a,428191000124101,Documentation of current medications,612.12,NaN,NaN
70,2011-04-15,3f33dcdd-8838-4077-910c-9fe59aa68796,0f01a013-c894-4387-ba6d-354f40d87a18,428191000124101,Documentation of current medications,483.63,NaN,NaN
173,2018-07-17,690e02eb-ed39-4d43-b9e9-44ee91b931f6,34a34165-2252-4e55-80a7-473ab7a1be53,118001005,Streptococcus pneumoniae group B antigen test,2179.01,72892002.0,Normal pregnancy


In [10]:
encounters.head()

,id,dateStart,dateStop,patient,provider,encounterClass,code,description,cost,reasonCode,reasonDescription
37319,530fd72c-d063-4263-add9-833fe9497299,2011-07-10T09:54:17Z,2011-07-10T10:09:17Z,1f6c814e-40d5-4d09-b96d-a32e61e8cc90,ef58ea08-d883-3957-8300-150554edc8fb,ambulatory,185345009,Encounter for symptom,129.16,444814009.0,Viral sinusitis (disorder)
9735,41e4b13a-a5f6-4dfb-8164-2d244423e049,2011-01-09T21:59:40Z,2011-01-09T22:14:40Z,0c9435eb-d5a7-4ee3-9769-baadca06765b,d78e84ec-30aa-3bba-a33a-f29a3a454662,outpatient,185347001,Encounter for problem (procedure),129.16,NaN,NaN
26799,dea845bc-46ff-4a7e-bdcd-4596dae925c6,2014-06-23T05:13:44Z,2014-06-23T05:43:44Z,2f9b2362-b5c9-49d2-960c-efae3710e900,d692e283-0833-3201-8e55-4f868a9c0736,wellness,185349003,Encounter for check up (procedure),129.16,NaN,NaN
2829,ee70d6f8-5c5d-4c5d-99ad-ef20ddc06c19,2011-04-19T04:32:00Z,2011-04-19T05:02:00Z,7d275ce4-032c-4af0-8998-4f79fa12d274,4bdaa4c2-c664-3089-aee2-7137abbad27f,wellness,185349003,Encounter for check up (procedure),129.16,NaN,NaN
1878,e1cfdaaf-da77-4004-abf8-ffe8e439f434,2004-08-27T06:45:35Z,2004-08-27T07:00:35Z,a5e81695-0475-48e1-ae41-6f04d89a6ca7,23834663-ed53-3da9-b330-d6e1ecb8428e,ambulatory,185347001,Encounter for problem,129.16,126906006.0,Neoplasm of prostate


Figure out how you would go about finding what patients have seen a given provider. Look in the encounters table, obviously. But what about if you wanted to know the patient's birthdate and medications, along with what provider they are seeing? Thats like 2 joins between 3 tables. On an enterprise scale of patients, this is *really* slow. Lets model these tables into a graph database.

# Creating Vertices
We are going to create vertices for our data to reside on. We will then join these vertices together with edges.

In [11]:
print(conn.gsql('''
    CREATE VERTEX Patient (PRIMARY_ID Id STRING, FirstName STRING, LastName STRING, Prefix STRING, Suffix STRING, MaidenName STRING, MaritalStatus STRING, Gender STRING, Race STRING, Ethnicity STRING, DateOfBirth DATETIME, DateOfDeath DATETIME DEFAULT "2999-12-31 00:00:00", SSN STRING, DL STRING, Passport STRING )
''', options=[]))

Trying version: v3_0_5
Connecting to syntheapytg.i.tgcloud.io:14240
If there is any relative path, it is relative to <System.AppRoot>/dev/gdk/gsql
The vertex type Patient is created.



In [12]:
print(conn.gsql('''
    CREATE VERTEX Provider (PRIMARY_ID Id STRING, Name STRING, Gender STRING, Speciality STRING, Utilization INT)
''', options=[]))

Trying version: v3_0_5
Connecting to syntheapytg.i.tgcloud.io:14240
If there is any relative path, it is relative to <System.AppRoot>/dev/gdk/gsql
The vertex type Provider is created.



In [13]:
print(conn.gsql('''
    CREATE VERTEX Condition (PRIMARY_ID Id STRING, Code STRING, Description STRING)
''', options=[]))

Trying version: v3_0_5
Connecting to syntheapytg.i.tgcloud.io:14240
If there is any relative path, it is relative to <System.AppRoot>/dev/gdk/gsql
The vertex type Condition is created.



In [14]:
print(conn.gsql('''
    CREATE VERTEX Medication(PRIMARY_ID Id STRING, Code STRING, Description STRING, Cost DOUBLE, Dispense INT, TotalCost DOUBLE, ReasonCode STRING, ReasonDescription STRING)
''', options=[]))

Trying version: v3_0_5
Connecting to syntheapytg.i.tgcloud.io:14240
If there is any relative path, it is relative to <System.AppRoot>/dev/gdk/gsql
The vertex type Medication is created.



In [15]:
print(conn.gsql('''
    CREATE VERTEX PatientProcedure(PRIMARY_ID Id STRING, Code STRING, Description STRING, Cost DOUBLE, ReasonCode STRING, ReasonDescription STRING)
''', options=[]))

Trying version: v3_0_5
Connecting to syntheapytg.i.tgcloud.io:14240
If there is any relative path, it is relative to <System.AppRoot>/dev/gdk/gsql
The vertex type PatientProcedure is created.



In [16]:
print(conn.gsql('''
    CREATE VERTEX Visit (PRIMARY_ID Id STRING, Code STRING, Description STRING, Started DATETIME, Stopped DATETIME, Cost DOUBLE) 
''', options=[]))

Trying version: v3_0_5
Connecting to syntheapytg.i.tgcloud.io:14240
If there is any relative path, it is relative to <System.AppRoot>/dev/gdk/gsql
The vertex type Visit is created.



# Adding Edges
Now here comes what makes a graph database special - predetermined relationships between different entities. We are going to add *edges* between are vertices, so that we don't have to perform joins when manipulating the data

In [17]:
# Create Edge Between Patients and Visits, Providers and Visits, and Patients and Providers

print(conn.gsql('''
    CREATE DIRECTED EDGE patientVisit (FROM Patient, TO Visit) WITH REVERSE_EDGE = "patientSeen"
''', options=[]))

print(conn.gsql('''
    CREATE DIRECTED EDGE providerVisit (FROM Provider, TO Visit) WITH REVERSE_EDGE = "providerSeen"
''', options=[]))

print(conn.gsql('''
    CREATE DIRECTED EDGE patientSeesProvider (FROM Patient, TO Provider) WITH REVERSE_EDGE = "providerSeesPatient"
''', options=[]))

Trying version: v3_0_5
Connecting to syntheapytg.i.tgcloud.io:14240
If there is any relative path, it is relative to <System.AppRoot>/dev/gdk/gsql
The edge type patientVisit is created.
The reverse edge type patientSeen is created.

Trying version: v3_0_5
Connecting to syntheapytg.i.tgcloud.io:14240
If there is any relative path, it is relative to <System.AppRoot>/dev/gdk/gsql
The edge type providerVisit is created.
The reverse edge type providerSeen is created.

Trying version: v3_0_5
Connecting to syntheapytg.i.tgcloud.io:14240
If there is any relative path, it is relative to <System.AppRoot>/dev/gdk/gsql
The edge type patientSeesProvider is created.
The reverse edge type providerSeesPatient is created.



In [18]:
# Create edge between Patients and Medications

print(conn.gsql('''
    CREATE DIRECTED EDGE hasMedication (FROM Patient, TO Medication, Started DATETIME, Stopped DATETIME DEFAULT "2999-12-31 00:00:00") WITH REVERSE_EDGE = "patientMedication"
''', options=[]))

Trying version: v3_0_5
Connecting to syntheapytg.i.tgcloud.io:14240
If there is any relative path, it is relative to <System.AppRoot>/dev/gdk/gsql
The edge type hasMedication is created.
The reverse edge type patientMedication is created.



In [19]:
# Create edge between Patients and Conditions

print(conn.gsql('''
    CREATE DIRECTED EDGE hasCondition (FROM Patient, TO Condition, Started DATETIME, Stopped DATETIME DEFAULT "2999-12-31 00:00:00") WITH REVERSE_EDGE = "patientCondition"
''', options=[]))

Trying version: v3_0_5
Connecting to syntheapytg.i.tgcloud.io:14240
If there is any relative path, it is relative to <System.AppRoot>/dev/gdk/gsql
The edge type hasCondition is created.
The reverse edge type patientCondition is created.



In [20]:
# Create edge between Patients and Procedures as well as Visits and Procedure

print(conn.gsql('''
    CREATE DIRECTED EDGE hasProcedure (FROM Patient, TO PatientProcedure, Performed DATETIME) WITH REVERSE_EDGE = "patientProcedure"
''', options=[]))

print(conn.gsql('''
    CREATE DIRECTED EDGE procedureDuringVisit (FROM PatientProcedure, TO Visit, Performed DATETIME) WITH REVERSE_EDGE = "visitForProcedure"
''', options=[]))

Trying version: v3_0_5
Connecting to syntheapytg.i.tgcloud.io:14240
If there is any relative path, it is relative to <System.AppRoot>/dev/gdk/gsql
The edge type hasProcedure is created.
The reverse edge type patientProcedure is created.

Trying version: v3_0_5
Connecting to syntheapytg.i.tgcloud.io:14240
If there is any relative path, it is relative to <System.AppRoot>/dev/gdk/gsql
The edge type procedureDuringVisit is created.
The reverse edge type visitForProcedure is created.



# Tying it all together
After this, visit [here](http://localhost:14240/#/schema-designer) to see the completed schema

In [21]:
print(conn.gsql("ls", options=[]))

Trying version: v3_0_5
Connecting to syntheapytg.i.tgcloud.io:14240
If there is any relative path, it is relative to <System.AppRoot>/dev/gdk/gsql
---- Global vertices, edges, and all graphs
Vertex Types: 
  - VERTEX Patient(PRIMARY_ID Id STRING, FirstName STRING, LastName STRING, Prefix STRING, Suffix STRING, MaidenName STRING, MaritalStatus STRING, Gender STRING, Race STRING, Ethnicity STRING, DateOfBirth DATETIME, DateOfDeath DATETIME DEFAULT "32503593600", SSN STRING, DL STRING, Passport STRING) WITH STATS="OUTDEGREE_BY_EDGETYPE"
  - VERTEX Provider(PRIMARY_ID Id STRING, Name STRING, Gender STRING, Speciality STRING, Utilization INT) WITH STATS="OUTDEGREE_BY_EDGETYPE"
  - VERTEX Condition(PRIMARY_ID Id STRING, Code STRING, Description STRING) WITH STATS="OUTDEGREE_BY_EDGETYPE"
  - VERTEX Medication(PRIMARY_ID Id STRING, Code STRING, Description STRING, Cost DOUBLE, Dispense INT, TotalCost DOUBLE, ReasonCode STRING, ReasonDescription STRING) WITH STATS="OUTDEGREE_BY_EDGETYPE"
  - VE

In [22]:
print(conn.gsql('''
    CREATE GRAPH Synthea(Patient, Provider, Condition, Medication, PatientProcedure, Visit, patientVisit, patientSeen, providerVisit, providerSeen, patientSeesProvider, providerSeesPatient, hasMedication, patientMedication, hasCondition, patientCondition, hasProcedure, patientProcedure, procedureDuringVisit, visitForProcedure)
''', options=[]))

Trying version: v3_0_5
Connecting to syntheapytg.i.tgcloud.io:14240
If there is any relative path, it is relative to <System.AppRoot>/dev/gdk/gsql
Stopping GPE GSE RESTPP
Successfully stopped GPE GSE RESTPP in 0.006 seconds
Starting GPE GSE RESTPP
Successfully started GPE GSE RESTPP in 0.099 seconds
The graph Synthea is created.



# Loading Data
We could define loading jobs, but the easiest way is to use pyTigerGraph's loading functions.

In [43]:
# First, need to set graph name in connection

conn.graphname = "Synthea"
if cloud:
    conn.apiToken = conn.getToken(conn.createSecret())

In [44]:
patients.head()

,id,birthDate,deathDate,ssn,drivers,passport,prefix,first,last,suffix,maiden,marital,race,ethnicity,gender,birthPlace,address,city,state,zip
0,35e0c202-7d79-40cb-84f1-1b69e6f6be3c,1985-12-17,2999-12-31 00:00:00,999-59-3229,S99985207,X13132187X,Mr.,Ignacio928,Candelaria844,,,M,black,dominican,M,Portsmouth Saint John Parish DM,940 Cummerata Pathway Apt 23,Worcester,Massachusetts,1545
1,ffd2b713-518b-4660-b112-e1ee298ac8d1,1990-10-20,2999-12-31 00:00:00,999-77-6505,S99942717,X36715867X,Mr.,Robbie31,Parisian75,,,M,black,west_indian,M,Cheshire Massachusetts US,698 Cummerata Rapid,Boston,Massachusetts,2108
2,5311bb0b-a96c-4bab-8713-bc9405c420ee,1983-05-03,2999-12-31 00:00:00,999-50-9809,S99995282,X64495409X,Mr.,Sergio619,Dicki44,,,M,white,english,M,Malden Massachusetts US,761 Lubowitz Terrace,Malden,Massachusetts,2148
3,81b0149c-1b72-4e63-9ee4-62e4856fdd5c,1953-07-16,2999-12-31 00:00:00,999-12-3651,S99948962,X57816876X,Ms.,Cindie288,Hyatt152,,,S,white,french,F,Waltham Massachusetts US,1066 Weimann Extension,Brookline,Massachusetts,2215
4,d1f14152-8766-40cc-a53b-9e78aba41bc0,2002-08-02,2999-12-31 00:00:00,999-45-4207,S99955456,,,Hsiu839,Rutherford999,,,,white,french,F,Worcester Massachusetts US,622 Breitenberg Club Unit 79,Lowell,Massachusetts,1850


In [45]:
'''
CREATE VERTEX Patient (PRIMARY_ID Id STRING, FirstName STRING, LastName STRING, Prefix STRING, Suffix STRING, MaidenName STRING, MaritalStatus STRING, Gender STRING, Race STRING, Ethnicity STRING, DateOfBirth DATETIME, DateOfDeath DATETIME DEFAULT "2999-12-31 00:00:00", SSN STRING, DL STRING, Passport STRING )
'''

def upsertPatients(row):
    attributes = {
        "FirstName": row["first"],
        "LastName": row["last"], 
        "Prefix": row["prefix"],
        "Suffix": row["suffix"], 
        "MaidenName": row["maiden"],
        "MaritalStatus": row["marital"],
        "Gender": row["gender"],
        "Race": row["race"],
        "Ethnicity": row["ethnicity"],
        "DateOfBirth": row["birthDate"],
        "SSN": row["ssn"],
        "DL": row["drivers"],
        "Passport": row["passport"]
    }
    conn.upsertVertex("Patient", row["id"], attributes)

In [46]:
patients["deathDate"].fillna("2999-12-31 00:00:00", inplace=True)
patients.fillna("", inplace=True)

In [47]:
patients.apply(lambda row: upsertPatients(row), axis=1)

POST https://syntheapytg.i.tgcloud.io:9000/graph/Synthea => {"vertices": {"Patient": {"35e0c202-7d79-40cb-84f1-1b69e6f6be3c": {"FirstName": {"value": "Ignacio928"}, "LastName": {"value": "Candelaria844"}, "Prefix": {"value": "Mr."}, "Suffix": {"value": ""}, "MaidenName": {"value": ""}, "MaritalStatus": {"value": "M"}, "Gender": {"value": "M"}, "Race": {"value": "black"}, "Ethnicity": {"value": "dominican"}, "DateOfBirth": {"value": "1985-12-17"}, "SSN": {"value": "999-59-3229"}, "DL": {"value": "S99985207"}, "Passport": {"value": "X13132187X"}}}}}
https://syntheapytg.i.tgcloud.io:9000/graph/Synthea
[{'accepted_vertices': 1, 'accepted_edges': 0}]
POST https://syntheapytg.i.tgcloud.io:9000/graph/Synthea => {"vertices": {"Patient": {"ffd2b713-518b-4660-b112-e1ee298ac8d1": {"FirstName": {"value": "Robbie31"}, "LastName": {"value": "Parisian75"}, "Prefix": {"value": "Mr."}, "Suffix": {"value": ""}, "MaidenName": {"value": ""}, "MaritalStatus": {"value": "M"}, "Gender": {"value": "M"}, "Race

KeyboardInterrupt: 

In [ ]:
providers.head()

In [ ]:
# CREATE VERTEX Provider (PRIMARY_ID Id STRING, Name STRING, Gender STRING, Speciality STRING, Utilization INT)
def upsertProviders(row):
    attributes = {
        "Name": row["name"], 
        "Gender": row["gender"],
        "Speciality": row["specialty"],
        "Utilization": row["utilization"]
    }
    conn.upsertVertex("Provider", row["id"], attributes)

In [ ]:
providers.apply(lambda row: upsertProviders(row), axis=1)

In [ ]:
encounters.head()

In [ ]:
# CREATE VERTEX Visit (PRIMARY_ID Id STRING, Code STRING, Description STRING, Started DATETIME, Stopped DATETIME, Cost DOUBLE) 
def upsertVisits(row):
    attributes = {
        "Code": str(row["code"]),
        "Description": row["description"],
        "Started": row["dateStart"],
        "Stopped": row["dateStop"],
        "Cost": row["cost"]
    }

    conn.upsertVertex("Visit", row["id"], attributes)
    conn.upsertEdge("Patient", row["patient"], "patientVisit", "Visit", row["id"])
    conn.upsertEdge("Provider", row["provider"], "providerVisit", "Visit", row["id"])
    conn.upsertEdge("Patient", row["patient"], "patientSeesProvider", "Provider", row["provider"])

In [ ]:
encounters.apply(lambda row: upsertVisits(row), axis=1)

In [ ]:
medications.head()

In [ ]:
medications["dateEnd"].fillna("2999-12-31 00:00:00", inplace=True)
medications.fillna("", inplace=True)

In [ ]:
#CREATE VERTEX Medication(PRIMARY_ID Id STRING, Code STRING, Description STRING, Cost DOUBLE, Dispense INT, TotalCost DOUBLE, ReasonCode STRING, ReasonDescription STRING)
#CREATE DIRECTED EDGE hasMedication (FROM Patient, TO Medication, Started DATETIME, Stopped DATETIME DEFAULT "2999-12-31 00:00:00") WITH REVERSE_EDGE = "patientMedication"
def upsertMedications(row):
    attributes = {
        "Code": str(row["code"]),
        "Description": row["description"],
        "Cost": row["cost"],
        "Dispense": row["dispenses"],
        "TotalCost": row["totalCost"],
        "ReasonCode": str(row["reasonCode"]),
        "ReasonDescription": row["reasonDesc"]
    }

    conn.upsertVertex("Medication", row["code"], attributes)

    edgeAttributes = {
        "Started": row["dateStart"],
        "Stopped": row["dateEnd"]
    }

    conn.upsertEdge("Patient", row["patient"], "hasMedication", "Medication", row["code"])

In [ ]:
medications.apply(lambda row: upsertMedications(row), axis=1)

In [ ]:
conditions.head()

In [ ]:
len(conditions["condition"].unique())

In [ ]:
conditions.shape

In [ ]:
conditions["dateEnd"].fillna("2999-12-31 00:00:00", inplace=True)
conditions.fillna("", inplace=True)

In [ ]:
# CREATE VERTEX Condition (PRIMARY_ID Id STRING, Code STRING, Description STRING)
# CREATE DIRECTED EDGE hasCondition (FROM Patient, TO Condition, Started DATETIME, Stopped DATETIME DEFAULT "2999-12-31 00:00:00") WITH REVERSE_EDGE = "patientCondition"
def upsertConditions(row):
    attributes = {
        "Code": str(row["code"]),
        "Description": row["description"]
    }

    conn.upsertVertex("Condition", row["code"], attributes)
    conn.upsertEdge("Patient", row["patient"], "hasCondition", "Condition", row["code"])

In [ ]:
conditions.apply(lambda row: upsertConditions(row), axis=1)

In [ ]:
procedures.head()

In [ ]:
procedures.fillna("", inplace=True)

In [ ]:
# CREATE VERTEX PatientProcedure(PRIMARY_ID Id STRING, Code STRING, Description STRING, Cost DOUBLE, ReasonCode STRING, ReasonDescription STRING)
# CREATE DIRECTED EDGE hasProcedure (FROM Patient, TO PatientProcedure, Performed DATETIME) WITH REVERSE_EDGE = "patientProcedure"
# CREATE DIRECTED EDGE procedureDuringVisit (FROM PatientProcedure, TO Visit, Performed DATETIME) WITH REVERSE_EDGE = "visitForProcedure"

def upsertProcedure(row):
    attributes = {
        "Code": str(row["code"]),
        "Description": row["description"],
        "Cost": row["cost"],
        "ReasonCode": str(row["reasonCode"]),
        "ReasonDescription": row["reasonDescription"]
    }

    conn.upsertVertex("PatientProcedure", row["code"], attributes)

    edgeAttributes = {
        "Performed": row["date"]
    }

    conn.upsertEdge("Patient", row["patient"], "hasProcedure", "PatientProcedure", row["code"], edgeAttributes)
    conn.upsertEdge("PatientProcedure", row["code"], "procedureDuringVisit", "Visit", row["encounter"], edgeAttributes)

In [ ]:
procedures.apply(lambda row: upsertProcedure(row), axis=1)